# Méthode de back-translation

## Install & Packages

In [1]:
!pip install torch transformers
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.2

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import warnings
import os

# Désactiver les messages d'avertissement
warnings.filterwarnings("ignore")

## Rétrotraduction sur un texte

In [3]:
# Vérifier si CUDA est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""
  Cette fonction prend une chaîne de caractères en entrée et retourne une traduction rétrograde (back-translation) du texte en utilisant deux modèles de traduction automatique de Hugging Face. Le texte d'entrée est d'abord traduit du français vers l'anglais, puis de l'anglais vers le français.

  Input :
    text : str - Le texte à traduire
    src_lang : str - (facultatif) La langue source du texte (français par défaut)
    tgt_lang : str - (facultatif) La langue cible pour la traduction (anglais par défaut)

  Output :
    back_translated_text : str - La traduction rétrograde du texte en entrée
"""
def back_translation(text, src_lang="fr", tgt_lang="en"):
    # Charger les tokenizers et les modèles
    tokenizer_fr_to_en = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
    model_fr_to_en = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fr-en").to(device)
    
    tokenizer_en_to_fr = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
    model_en_to_fr = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-fr").to(device)

    # On découpe le texte par phrase
    sentences = text.split('. ')
    translated_chunks = []
    for chunk in tqdm(sentences, desc="Translating sentences"):
        # Traduire du français vers l'anglais
        inputs_fr_to_en = tokenizer_fr_to_en(chunk, return_tensors="pt").to(device)
        outputs_fr_to_en = model_fr_to_en.generate(**inputs_fr_to_en)
        translated_text_en = tokenizer_fr_to_en.decode(outputs_fr_to_en[0], skip_special_tokens=True)

        # Traduire de l'anglais vers le français
        inputs_en_to_fr = tokenizer_en_to_fr(translated_text_en, return_tensors="pt").to(device)
        outputs_en_to_fr = model_en_to_fr.generate(**inputs_en_to_fr)
        back_translated_text_fr = tokenizer_en_to_fr.decode(outputs_en_to_fr[0], skip_special_tokens=True)

        translated_chunks.append(back_translated_text_fr)

    back_translated_text = " ".join(translated_chunks)

    return back_translated_text



In [8]:
# Exemple d'utilisation
text = "Les travaux que nous avons engagés ont impliqué la participation de développeurs, d’un modélisateur 3D et multimédia mais aussi d’un ingénieur recherche et développement, révélant de ce fait le caractère R&D de notre projet. Notre participation en 2021 à des activités de veille technologique comme des congrès ou des salons confortent également cette position."
back_translated_text = back_translation(text)
print("\nTexte d'origine:", text)
print("Texte traduit en aller-retour:", back_translated_text)

Translating sentences: 100%|██████████| 2/2 [00:01<00:00,  1.53it/s]


Texte d'origine: Les travaux que nous avons engagés ont impliqué la participation de développeurs, d’un modélisateur 3D et multimédia mais aussi d’un ingénieur recherche et développement, révélant de ce fait le caractère R&D de notre projet. Notre participation en 2021 à des activités de veille technologique comme des congrès ou des salons confortent également cette position.
Texte traduit en aller-retour: Les travaux que nous avons entrepris ont impliqué la participation de développeurs, d'un modélisateur 3D et multimédia, mais aussi d'un ingénieur en recherche et développement, révélant ainsi le caractère de R-D de notre projet. Notre participation en 2021 à des activités de veille technologique telles que congrès et salons professionnels renforce également cette position.


## Rétrotraduction sur un fichier .txt

In [9]:
"""
  Cette fonction prend un chemin de fichier en entrée et retourne une traduction rétrograde (back-translation) du texte contenu dans le fichier en utilisant deux modèles de traduction automatique de Hugging Face. Le texte d'entrée est d'abord traduit du français vers l'anglais, puis de l'anglais vers le français.

  Input :
    file_path : str - Le chemin vers le fichier à traduire
    src_lang : str - (facultatif) La langue source du texte (français par défaut)
    tgt_lang : str - (facultatif) La langue cible pour la traduction (anglais par défaut)

  Output :
    str - Un message confirmant la création du fichier contenant la traduction rétrograde du texte en entrée.
"""
def back_translation_file(file_path, src_lang="fr", tgt_lang="en"):
    with open(file_path, "r") as fichier:
        contenu = fichier.read()
    
    back_translated_text = back_translation(contenu, src_lang, tgt_lang)
    
    # Déterminer le nom du fichier de sortie
    file_name, file_ext = os.path.splitext(file_path)
    file_ext = f"_back_translated_{tgt_lang}{file_ext}"
    file_out = file_name + file_ext

    with open(file_out, "w") as output_file:
        output_file.write(back_translated_text)

    return "Le fichier " + file_out + " a bien été crée."

In [10]:
back_translation_file("output.txt")

Translating sentences: 100%|██████████| 76/76 [01:27<00:00,  1.15s/it]


'Le fichier output_back_translated_en.txt a bien été crée.'